In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

from peft import PeftModel
from huggingface_hub import login
import torch
from mikkel_secrets import secrets

login(secrets["llama"]["token"])

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/mikkel/.cache/huggingface/token
Login successful


In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model_id = "meta-llama/Llama-2-7b-chat-hf"

compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #torch_dtype=torch.bfloat16, 
    quantization_config=quant_config,
    device_map={"": 0},
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:01<00:00,  1.44it/s]


In [13]:
ft_model = PeftModel.from_pretrained(model, "results/checkpoint-105")


text = """

Create a LinkedIn post using the language and tone of Mikkel Jensen. 
The post is about networking and it's importance. It includes two topics:
My meeting with Daniel, a technical project manager, and us talking freelance.
My meeting at proprty.ai, a startup predicting multiple aspects of buildings health, with the aim to reduce maintanance cost.

"""

inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = ft_model.generate(**inputs, max_new_tokens=500, max_time=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



Create a LinkedIn post using the language and tone of Mikkel Jensen. 
The post is about networking and it's importance. It includes two topics:
My meeting with Daniel, a technical project manager, and us talking freelance.
My meeting at proprty.ai, a startup predicting multiple aspects of buildings health, with the aim to reduce maintanance cost.


The post is written in english and has the hashtag #networking.


A few weeks ago I was fortunate enough to have two meetings that I believe are important for everyone to know about.

I met Daniel, a technical project manager, who was available for a chat and was interested in talking about freelancing.

We had a great conversation about the pros and cons of freelancing, how to find clients, how to stay motivated and how to handle the paperwork.

Freelancing is something I have been doing for a while now, and it’s great to have the opportunity to learn from others.


Later that day I visited Peter, the CEO of property.ai, a startup that ha